# Simulate a 2D sensor with realistic sizes. 

In [1]:
import numpy as np
from scipy.interpolate import interp2d, SmoothBivariateSpline, RectBivariateSpline
import matplotlib.pyplot as plt
from matplotlib import cm

w, h = 12, 12 # sensor size in centimeters
p = (w/2+2 , h/2-3 , 1.75) # press position and depth in centimeters

N = 6 # data points (sensors) in each dimension
xs = np.linspace(0,w,N+2)[1:-1] # sensor position x in centimeters
ys = np.linspace(0,h,N+2)[1:-1] # sensor position y in centimeters

z = np.zeros((3,3))
z[1,1] = p[2]
f = interp2d([0, p[0], w], [0, p[1], h], z, kind='linear')
zs = f( xs, ys )
xs, ys = np.meshgrid(xs, ys)

## display with plotly

In [2]:
import plotly.graph_objects as go

# Create figure
fig = go.Figure(data=
    [go.Scatter3d(x=xs.flatten(), y=ys.flatten(), z=zs.flatten(), mode="markers", marker=dict(color="red", size=4))]
)

fig.update_layout(
    scene=dict(
        xaxis=dict(            range=[0, w],        ),
        yaxis=dict(            range=[0, h],        ),
        zaxis=dict(            range=[0, p[2]],        ),
    )
)

fig.show()

## Resconstruct the peak (pressing) position from the simulated data by using the z value (height) of each sensor.

In [3]:
from scipy.optimize import minimize

def min_fun(p, xs, ys, zs, w,h) -> float:
    z = np.zeros((3,3))
    z[1,1] = p[2]
    f = interp2d([0, p[0], w], [0, p[1], h], z, kind='linear')
    zs_ = np.zeros_like(zs.flatten())
    for i in range(len(zs_)):
        zs_[i] = f(xs.flatten()[i], ys.flatten()[i])

    return np.sum(((zs.flatten() - zs_)**2))

# test the minimization function
print(min_fun(p, xs, ys, zs, w,h))
print(f"ground truth at {p}.")

res = minimize(min_fun, x0=[xs[1,1],ys[1,1],zs[1,1]], args=(xs, ys, zs, w,h))
print(f"optimization at {res.x} with an error of {res.fun}.")

# the starting point (x0) must be within the range of the sensors! Otherwise the minimization will fail.

0.0
ground truth at (8.0, 3.0, 1.75).
optimization at [7.99999982 3.00000685 1.7499988 ] with an error of 2.0386050508737898e-11.


## use only a subset of the sensors to reconstruct the peak position

In [4]:
import random

# only a subset of the sensors are used for the minimization.
# When selecting a subset of the sensors it can happen that they are only on one side of the pressure point
# thus the minimization will not work.

k = 4
subset = random.choices(range(N*N), k=k)

res = minimize(min_fun, x0=[xs[1,1],ys[1,1],zs[1,1]], args=(xs.flatten()[subset], ys.flatten()[subset], zs.flatten()[subset], w,h))
print(res)

      fun: 0.07527601204416214
 hess_inv: array([[10.30472128, -5.94316147,  1.54094845],
       [-5.94316147,  4.70204445, -1.02825411],
       [ 1.54094845, -1.02825411,  1.17214331]])
      jac: array([-9.31322575e-10, -9.31322575e-10,  0.00000000e+00])
  message: 'Optimization terminated successfully.'
     nfev: 76
      nit: 15
     njev: 19
   status: 0
  success: True
        x: array([7.02040812, 1.12723548, 1.8552733 ])


# Adding noise to the simulated data to verify how robust the reconstruction is.

In [5]:
print(f"ground truth at {p}.")

std = 0.1
nx, ny, nz = np.random.normal(0,std,xs.shape), np.random.normal(0,std,ys.shape), np.random.normal(0,std,zs.shape)

res = minimize(min_fun, x0=[xs[1,1],ys[1,1],zs[1,1]], args=(xs+nx, ys+ny, zs+nz, w,h))
print(f"optimization with a noise level (std={std}) at {res.x} with an error of {res.fun}.")

ground truth at (8.0, 3.0, 1.75).
optimization with a noise level (std=0.1) at [8.04044015 2.78534255 1.77766048] with an error of 0.31679187493348326.
